## Data Population Script

#### 1. Data Source: Ecommerce transactions -> https://www.kaggle.com/datasets/smayanj/e-commerce-transactions-dataset
#### 2. Data Source: Chocolate Sales Data -> https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales 
#### 3. Data Source: Customer Transactions Dataset -> https://www.kaggle.com/datasets/mohammedarfathr/customer-transactions-dataset 
--------------------------

Installing packages

In [1]:
with open('./requirements.txt', 'r') as file:
    packages = file.readlines()
    
for package in packages:
    package = package.strip()  # Remove whitespace and newlines
    if package:
        print(f'Installing {package}...')
        !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org {package}

Installing sqlalchemy...
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Installing psycopg2-binary...
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Installing kagglehub...
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Installing dotenv...
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Installing pandas...
Defaulting to user installation because normal site-package

Generating connection with PostgreSql

In [43]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect

# Load environment variables from .env file
load_dotenv()

# Specify the path to the .env file
env_path = os.path.join(os.path.dirname(os.getcwd()), '.env')

# Load environment variables from specified .env file
load_dotenv(env_path)

# Get database connection parameters from environment variables
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_name = os.getenv('POSTGRES_DB')
db_host = 'localhost'  # Since we're connecting to a container
db_port = '5432'

# Create the database connection URL
db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create the SQLAlchemy engine
engine = create_engine(db_url)

# Create inspector to check for existing tables
inspector = inspect(engine)

# Test the connection
try:
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
except Exception as e:
    print(f"Connection failed: {str(e)}")

Connection successful!


In [44]:
from sqlalchemy import Table, Column, Integer, String, Float, Date, MetaData, text
from sqlalchemy.dialects.postgresql import UUID
import uuid

# Create a MetaData instance
metadata = MetaData()

# Function to create table if it doesn't exist
def create_table_if_not_exists(table_name, table_definition):
    if not inspector.has_table(table_name):
        table_definition.create(engine)
        print(f"Table {table_name} created successfully")
    else:
        print(f"Table {table_name} already exists")

# Define the tables
Ecommerce_Transactions = Table(
    'Ecommerce_Transactions', 
    metadata,
    Column('id', UUID(as_uuid=True), primary_key=True, server_default=text("gen_random_uuid()")),
    Column('Transaction_ID', Integer),
    Column('User_Name', String(100), nullable=False),
    Column('Age', Integer, nullable=False),
    Column('Country', String(50), nullable=False),
    Column('Product_Category', String(50), nullable=False),
    Column('Purchase_Amount', Float, nullable=False),
    Column('Payment_Method', String(50), nullable=False),
    Column('Transaction_Date', Date, nullable=False)
)

Chocolate_Sales_Data = Table(
    'Chocolate_Sales_Data',
    metadata,
    Column('id', UUID(as_uuid=True), primary_key=True, server_default=text("gen_random_uuid()")),
    Column('Sales_Person', String(100), nullable=False),
    Column('Country', String(50), nullable=False),
    Column('Product', String(100), nullable=False),
    Column('Date', Date, nullable=False),
    Column('Amount', String(15), nullable=False),
    Column('Boxes_Shipped', Integer, nullable=False)
)

Customer_Transactions_Dataset = Table(
    'Customer_Transactions_Dataset',
    metadata,
    Column('id', UUID(as_uuid=True), primary_key=True, server_default=text("gen_random_uuid()")),
    Column('Customer_ID', Integer, nullable=True),
    Column('Age', Float, nullable=True),
    Column('Annual_Income', Float, nullable=False),
    Column('Spending_Score', Float, nullable=False),
    Column('Purchase_Frequency', Integer, nullable=False),
    Column('Transaction_Amount', Float, nullable=True)
)

# Create tables if they don't exist
create_table_if_not_exists('Ecommerce_Transactions', Ecommerce_Transactions)
create_table_if_not_exists('Chocolate_Sales_Data', Chocolate_Sales_Data)
create_table_if_not_exists('Customer_Transactions_Dataset', Customer_Transactions_Dataset)

Table Ecommerce_Transactions created successfully
Table Chocolate_Sales_Data created successfully
Table Customer_Transactions_Dataset created successfully


Loading data from CSV files and uploading to PostgreSQL

In [47]:
import pandas as pd
import os

# Function to load and upload data
def load_and_upload_data(file_path, table_name, engine):
    try:
        # Read CSV file
        df = pd.read_csv(file_path)
        
        # Get first 10 rows
        df_sample = df.head(10)
        
        # Handle date columns based on table name
        if table_name == 'Ecommerce_Transactions':
            df_sample['Transaction_Date'] = pd.to_datetime(df_sample['Transaction_Date'])
        elif table_name == 'Chocolate_Sales_Data':
            df_sample['Date'] = pd.to_datetime(df_sample['Date'])
        
        # Upload to PostgreSQL
        df_sample.to_sql(table_name, engine, if_exists='append', index=False)
        print(f'Successfully uploaded {len(df_sample)} rows to {table_name}')
        
    except Exception as e:
        print(f'Error processing {table_name}: {str(e)}')

# Define file paths
data_files = {
    'Ecommerce_Transactions': 'data/ecommerce_transactions.csv',
    'Chocolate_Sales_Data': 'data/Chocolate Sales.csv',
    'Customer_Transactions_Dataset': 'data/unclean_customer_data.csv'
}

# Process each file
for table_name, file_path in data_files.items():
    if os.path.exists(file_path):
        load_and_upload_data(file_path, table_name, engine)
    else:
        print(f'File not found: {file_path}')

Successfully uploaded 10 rows to Ecommerce_Transactions
Successfully uploaded 10 rows to Chocolate_Sales_Data
Successfully uploaded 10 rows to Customer_Transactions_Dataset


/var/folders/lk/b67y2x_537sd_nwg8my4ghp40000gn/T/ipykernel_25536/1667415610.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['Transaction_Date'] = pd.to_datetime(df_sample['Transaction_Date'])
/var/folders/lk/b67y2x_537sd_nwg8my4ghp40000gn/T/ipykernel_25536/1667415610.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sample['Date'] = pd.to_datetime(df_sample['Date'])
/var/folders/lk/b67y2x_537sd_nwg8my4ghp40000gn/T/ipykernel_25536/1667415610.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Verify data upload by querying tables

In [48]:
# Verify uploaded data
def verify_data(table_name, engine):
    query = f'SELECT count(*) FROM public."{table_name}" LIMIT 5'
    try:
        result = pd.read_sql(query, engine)
        print(f'\nData preview for {table_name}:')
        print(result)
    except Exception as e:
        print(f'Error querying {table_name}: {str(e)}')

# Verify each table
for table_name in data_files.keys():
    verify_data(table_name, engine)


Data preview for Ecommerce_Transactions:
   count
0     20

Data preview for Chocolate_Sales_Data:
   count
0     20

Data preview for Customer_Transactions_Dataset:
   count
0     20
